**IMPORTING ALL DEPENDENCIES**

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import  RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
gb_regressor = GradientBoostingRegressor()
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor

**loading dataset from csv**

In [2]:

train=pd.read_csv('train.csv')

In [3]:
test=pd.read_csv('/content/drive/MyDrive/datasets/test.csv')

In [3]:
meal=pd.read_csv('meal_info.csv')

In [4]:
train=train.merge(meal, on='meal_id')

In [5]:
train.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,category,cuisine
0,1379560,1,55,1885,136.83,152.29,0,0,177,Beverages,Thai
1,1040403,1,24,1885,136.83,136.83,0,0,1498,Beverages,Thai
2,1103215,1,11,1885,136.83,136.83,0,0,959,Beverages,Thai
3,1034383,1,83,1885,121.31,120.31,0,1,1025,Beverages,Thai
4,1118999,1,32,1885,114.52,113.52,0,1,445,Beverages,Thai


In [6]:
center_id = 55
meal_id = 1993

In [7]:
def preprocessreg(train):
  train_df = train[train['center_id']==center_id]
  train_df = train_df[train_df['meal_id']==meal_id]

  categorical_cols = ['category', 'cuisine']
  from sklearn.preprocessing import LabelEncoder
  label_encoder = LabelEncoder()
  for column in categorical_cols:
    train_df[column] = label_encoder.fit_transform(train_df[column])

  train_df.isnull().sum()

  train_df = train_df.reset_index()

  return train_df




train_df=preprocessreg(train)

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  145 non-null    int64  
 1   id                     145 non-null    int64  
 2   week                   145 non-null    int64  
 3   center_id              145 non-null    int64  
 4   meal_id                145 non-null    int64  
 5   checkout_price         145 non-null    float64
 6   base_price             145 non-null    float64
 7   emailer_for_promotion  145 non-null    int64  
 8   homepage_featured      145 non-null    int64  
 9   num_orders             145 non-null    int64  
 10  category               145 non-null    int32  
 11  cuisine                145 non-null    int32  
dtypes: float64(2), int32(2), int64(8)
memory usage: 12.6 KB


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [10]:
train_df.head()

,index,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,category,cuisine
0,11092,1466964,1,55,1993,136.83,135.83,0,0,270,0,0
1,11169,1035555,2,55,1993,133.89,133.89,0,0,121,0,0
2,11246,1282652,3,55,1993,134.86,133.86,0,0,258,0,0
3,11323,1066744,4,55,1993,134.89,133.89,0,0,82,0,0
4,11400,1115779,5,55,1993,147.50,145.50,0,0,81,0,0


#**XB REGRESSOR**

In [20]:
# xb regressor
def xgbtrain(train_df):
  # Train-test split
  import pandas as pd
  from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
  from xgboost import XGBRegressor
  from sklearn import metrics
  X = train_df.drop(columns=['num_orders', 'id', 'meal_id', 'center_id', 'index'], axis=1)
  Y = train_df['num_orders']
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

  # Define the XGBoost model
  model = XGBRegressor()

  # Train the model without hyperparameter tuning
  model.fit(X_train, Y_train)
  training_data_prediction = model.predict(X_train)
  test_data_prediction = model.predict(X_test)
  score_2 = metrics.mean_absolute_error(Y_test, test_data_prediction)
  print('Mean Absolute Error (No Hyperparameter Tuning): ', score_2)

  # Hyperparameter tuning using Grid Search
  param_grid = {
      'learning_rate': [0.01, 0.1, 0.2],
      'n_estimators': [100, 200, 300],
      'max_depth': [3, 4, 5]
  }

  grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_error')
  grid_search.fit(X_train, Y_train)

  best_params_grid = grid_search.best_params_
  print('Best Parameters (Grid Search):', best_params_grid)

  # Train the model with the best parameters from Grid Search
  model_grid = XGBRegressor(**best_params_grid)
  model_grid.fit(X_train, Y_train)
  test_data_prediction_grid = model_grid.predict(X_test)
  score_grid = metrics.mean_absolute_error(Y_test, test_data_prediction_grid)
  print('Mean Absolute Error (Grid Search):', score_grid)

  # Hyperparameter tuning using Random Search
  param_dist = {
      'learning_rate': [0.01, 0.1, 0.2],
      'n_estimators': [100, 200, 300],
      'max_depth': [3, 4, 5]
  }

  random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3,
                                      scoring='neg_mean_absolute_error', random_state=42)
  random_search.fit(X_train, Y_train)

  best_params_random = random_search.best_params_
  print('Best Parameters (Random Search):', best_params_random)

  # Train the model with the best parameters from Random Search
  model_random = XGBRegressor(**best_params_random)
  model_random.fit(X_train, Y_train)
  test_data_prediction_random = model_random.predict(X_test)
  score_random = metrics.mean_absolute_error(Y_test, test_data_prediction_random)
  print('Mean Absolute Error (Random Search):', score_random)

xgbtrain(train_df)


Mean Absolute Error (No Hyperparameter Tuning):  94.08653680209456
Best Parameters (Grid Search): {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300}
Mean Absolute Error (Grid Search): 78.68997244999326
Best Parameters (Random Search): {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.1}
Mean Absolute Error (Random Search): 83.04240733179553


#**RANDOM FOREST**

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

def random_forest_train(train_df):
    # Train-test split
    X = train_df.drop(columns=['num_orders', 'id', 'meal_id', 'center_id', 'index'], axis=1)
    Y = train_df['num_orders']
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

    # Define the Random Forest model
    rf_model = RandomForestRegressor()

    # Train the model without hyperparameter tuning
    rf_model.fit(X_train, Y_train)
    training_data_prediction_rf = rf_model.predict(X_train)
    test_data_prediction_rf = rf_model.predict(X_test)
    score_rf_2 = metrics.mean_absolute_error(Y_test, test_data_prediction_rf)
    print('Mean Absolute Error (No Hyperparameter Tuning): ', score_rf_2)

    # Hyperparameter tuning using Grid Search
    param_grid_rf = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20,30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, cv=3, scoring='neg_mean_absolute_error')
    grid_search_rf.fit(X_train, Y_train)

    best_params_grid_rf = grid_search_rf.best_params_
    print('Best Parameters (Grid Search):', best_params_grid_rf)

    # Train the model with the best parameters from Grid Search
    rf_model_grid = RandomForestRegressor(**best_params_grid_rf)
    rf_model_grid.fit(X_train, Y_train)
    test_data_prediction_grid_rf = rf_model_grid.predict(X_test)
    score_grid_rf = metrics.mean_absolute_error(Y_test, test_data_prediction_grid_rf)
    print('Mean Absolute Error (Grid Search):', score_grid_rf)

    # Hyperparameter tuning using Random Search
    param_dist_rf = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    random_search_rf = RandomizedSearchCV(estimator=rf_model, param_distributions=param_dist_rf, n_iter=10, cv=3,
                                          scoring='neg_mean_absolute_error', random_state=42)
    random_search_rf.fit(X_train, Y_train)

    best_params_random_rf = random_search_rf.best_params_
    print('Best Parameters (Random Search):', best_params_random_rf)

    # Train the model with the best parameters from Random Search
    rf_model_random = RandomForestRegressor(**best_params_random_rf)
    rf_model_random.fit(X_train, Y_train)
    test_data_prediction_random_rf = rf_model_random.predict(X_test)
    score_random_rf = metrics.mean_absolute_error(Y_test, test_data_prediction_random_rf)
    print('Mean Absolute Error (Random Search):', score_random_rf)

random_forest_train(train_df)

Mean Absolute Error (No Hyperparameter Tuning):  73.3093103448276
Best Parameters (Grid Search): {'max_depth': 30, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
Mean Absolute Error (Grid Search): 71.6968750765518
Best Parameters (Random Search): {'n_estimators': 50, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': None}
Mean Absolute Error (Random Search): 69.35076683087026


#**DECISION TREE**

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

def decision_tree_train(train_df):
    # Train-test split
    X = train_df.drop(columns=['num_orders', 'id', 'meal_id', 'center_id', 'index'], axis=1)
    Y = train_df['num_orders']
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

    # Define the Decision Tree model
    model = DecisionTreeRegressor()

    # Train the model without hyperparameter tuning
    model.fit(X_train, Y_train)
    training_data_prediction = model.predict(X_train)
    test_data_prediction = model.predict(X_test)
    score_2 = metrics.mean_absolute_error(Y_test, test_data_prediction)
    print('Mean Absolute Error (No Hyperparameter Tuning): ', score_2)

    # Hyperparameter tuning using Grid Search
    param_grid = {
        'max_depth': [None, 5, 10, 15,20,30],
        'min_samples_split': [2, 5, 10,15,20],
        'min_samples_leaf': [1, 2, 4,5]
    }

    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_error')
    grid_search.fit(X_train, Y_train)

    best_params_grid = grid_search.best_params_
    print('Best Parameters (Grid Search):', best_params_grid)

    # Train the model with the best parameters from Grid Search
    model_grid = DecisionTreeRegressor(**best_params_grid)
    model_grid.fit(X_train, Y_train)
    test_data_prediction_grid = model_grid.predict(X_test)
    score_grid = metrics.mean_absolute_error(Y_test, test_data_prediction_grid)
    print('Mean Absolute Error (Grid Search):', score_grid)

    # Hyperparameter tuning using Random Search
    param_dist = {
        'max_depth': [None, 5, 10, 15],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3,
                                       scoring='neg_mean_absolute_error', random_state=42)
    random_search.fit(X_train, Y_train)

    best_params_random = random_search.best_params_
    print('Best Parameters (Random Search):', best_params_random)

    # Train the model with the best parameters from Random Search
    model_random = DecisionTreeRegressor(**best_params_random)
    model_random.fit(X_train, Y_train)
    test_data_prediction_random = model_random.predict(X_test)
    score_random = metrics.mean_absolute_error(Y_test, test_data_prediction_random)
    print('Mean Absolute Error (Random Search):', score_random)

# Call the function with your train_df
# decision_tree_train(train_df)
decision_tree_train(train_df)

Mean Absolute Error (No Hyperparameter Tuning):  100.96551724137932
Best Parameters (Grid Search): {'max_depth': None, 'min_samples_leaf': 5, 'min_samples_split': 20}
Mean Absolute Error (Grid Search): 66.90671624971827
Best Parameters (Random Search): {'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 5}
Mean Absolute Error (Random Search): 68.67422003284072


#**GRADIENT BOOSTING REGRESSOR**

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics

def gbtrain(train_df):
    # Train-test split
    X = train_df.drop(columns=['num_orders', 'id', 'meal_id', 'center_id', 'index'], axis=1)
    Y = train_df['num_orders']
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

    # Define the GradientBoostingRegressor model
    model = GradientBoostingRegressor()

    # Train the model without hyperparameter tuning
    model.fit(X_train, Y_train)
    training_data_prediction = model.predict(X_train)
    test_data_prediction = model.predict(X_test)
    score_2 = metrics.mean_absolute_error(Y_test, test_data_prediction)
    print('Mean Absolute Error (No Hyperparameter Tuning): ', score_2)

    # Hyperparameter tuning using Grid Search
    param_grid = {
        'learning_rate': [0.01, 0.1, 0.2,0.3],
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 4, 5,6]
    }

    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_error')
    grid_search.fit(X_train, Y_train)

    best_params_grid = grid_search.best_params_
    print('Best Parameters (Grid Search):', best_params_grid)

    # Train the model with the best parameters from Grid Search
    model_grid = GradientBoostingRegressor(**best_params_grid)
    model_grid.fit(X_train, Y_train)
    test_data_prediction_grid = model_grid.predict(X_test)
    score_grid = metrics.mean_absolute_error(Y_test, test_data_prediction_grid)
    print('Mean Absolute Error (Grid Search):', score_grid)

    # Hyperparameter tuning using Random Search
    param_dist = {
        'learning_rate': [0.01, 0.1, 0.2],
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 4, 5]
    }

    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3,
                                       scoring='neg_mean_absolute_error', random_state=42)
    random_search.fit(X_train, Y_train)

    best_params_random = random_search.best_params_
    print('Best Parameters (Random Search):', best_params_random)

    # Train the model with the best parameters from Random Search
    model_random = GradientBoostingRegressor(**best_params_random)
    model_random.fit(X_train, Y_train)
    test_data_prediction_random = model_random.predict(X_test)
    score_random = metrics.mean_absolute_error(Y_test, test_data_prediction_random)
    print('Mean Absolute Error (Random Search):', score_random)

# Call the function with your train_df
# gbtrain(train_df)
gbtrain(train_df)

Mean Absolute Error (No Hyperparameter Tuning):  89.18861337468383
Best Parameters (Grid Search): {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200}
Mean Absolute Error (Grid Search): 83.77450189258033
Best Parameters (Random Search): {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.01}
Mean Absolute Error (Random Search): 79.94266919309469


# **ExtraTreesRegressor**

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import metrics

def extra_tree_train(train_df):
    # Train-test split
    X = train_df.drop(columns=['num_orders', 'id', 'meal_id', 'center_id', 'index'], axis=1)
    Y = train_df['num_orders']
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

    # Define the ExtraTreesRegressor model
    model = ExtraTreesRegressor()

    # Train the model without hyperparameter tuning
    model.fit(X_train, Y_train)
    training_data_prediction = model.predict(X_train)
    test_data_prediction = model.predict(X_test)
    score_2 = metrics.mean_absolute_error(Y_test, test_data_prediction)
    print('Mean Absolute Error (No Hyperparameter Tuning): ', score_2)

    # Hyperparameter tuning using Grid Search
    param_grid = {
        'n_estimators': [50, 100, 150],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_error')
    grid_search.fit(X_train, Y_train)

    best_params_grid = grid_search.best_params_
    print('Best Parameters (Grid Search):', best_params_grid)

    # Train the model with the best parameters from Grid Search
    model_grid = ExtraTreesRegressor(**best_params_grid)
    model_grid.fit(X_train, Y_train)
    test_data_prediction_grid = model_grid.predict(X_test)
    score_grid = metrics.mean_absolute_error(Y_test, test_data_prediction_grid)
    print('Mean Absolute Error (Grid Search):', score_grid)

    # Hyperparameter tuning using Random Search
    param_dist = {
        'n_estimators': [50, 100, 150],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3,
                                       scoring='neg_mean_absolute_error', random_state=42)
    random_search.fit(X_train, Y_train)

    best_params_random = random_search.best_params_
    print('Best Parameters (Random Search):', best_params_random)

    # Train the model with the best parameters from Random Search
    model_random = ExtraTreesRegressor(**best_params_random)
    model_random.fit(X_train, Y_train)
    test_data_prediction_random = model_random.predict(X_test)
    score_random = metrics.mean_absolute_error(Y_test, test_data_prediction_random)
    print('Mean Absolute Error (Random Search):', score_random)

# Call the function with your train_df
# extra_tree_train(train_df)
extra_tree_train(train_df)

Mean Absolute Error (No Hyperparameter Tuning):  73.45413793103448
Best Parameters (Grid Search): {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50}
Mean Absolute Error (Grid Search): 71.66937046061892
Best Parameters (Random Search): {'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_depth': None}
Mean Absolute Error (Random Search): 71.27766854341978


# **AdaBoostRegressort**

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import AdaBoostRegressor
from sklearn import metrics

def adaboost_train(train_df):
    # Train-test split
    X = train_df.drop(columns=['num_orders', 'id', 'meal_id', 'center_id', 'index'], axis=1)
    Y = train_df['num_orders']
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

    # Define the AdaBoostRegressor model
    model = AdaBoostRegressor()

    # Train the model without hyperparameter tuning
    model.fit(X_train, Y_train)
    training_data_prediction = model.predict(X_train)
    test_data_prediction = model.predict(X_test)
    score_2 = metrics.mean_absolute_error(Y_test, test_data_prediction)
    print('Mean Absolute Error (No Hyperparameter Tuning): ', score_2)

    # Hyperparameter tuning using Grid Search
    param_grid = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2]
    }

    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_error')
    grid_search.fit(X_train, Y_train)
    return grid_search
    best_params_grid = grid_search.best_params_
    print('Best Parameters (Grid Search):', best_params_grid)

    # Train the model with the best parameters from Grid Search
    model_grid = AdaBoostRegressor(**best_params_grid)
    model_grid.fit(X_train, Y_train)
    test_data_prediction_grid = model_grid.predict(X_test)
    score_grid = metrics.mean_absolute_error(Y_test, test_data_prediction_grid)
    print('Mean Absolute Error (Grid Search):', score_grid)

    # Hyperparameter tuning using Random Search
    param_dist = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2]
    }

    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3,
                                       scoring='neg_mean_absolute_error', random_state=42)
    random_search.fit(X_train, Y_train)

    best_params_random = random_search.best_params_
    print('Best Parameters (Random Search):', best_params_random)

    # Train the model with the best parameters from Random Search
    model_random = AdaBoostRegressor(**best_params_random)
    model_random.fit(X_train, Y_train)
    test_data_prediction_random = model_random.predict(X_test)
    score_random = metrics.mean_absolute_error(Y_test, test_data_prediction_random)
    print('Mean Absolute Error (Random Search):', score_random)

# Call the function with your train_df
# adaboost_train(train_df)
model=adaboost_train(train_df)

Mean Absolute Error (No Hyperparameter Tuning):  80.17630974684995


In [27]:
model.best_params_

{'learning_rate': 0.01, 'n_estimators': 50}